<!-- SPDX-License-Identifier: CC-BY-4.0 -->
<!-- Copyright Contributors to the ODPi Egeria project 2024. -->

![Egeria Logo](https://raw.githubusercontent.com/odpi/egeria/main/assets/img/ODPi_Egeria_Logo_color.png)

### Egeria Workbook

# Working with pyegeria

## Introduction

The `pyegeria` python library provides a wide-ranging set of python functions for interacting with the Egeria runtime.  In addition there is a command level interface (CLI) for issuing commands to Egeria and widgets to display information held by Egeria.

This notebook provides an introductory guide to `pyegeria`.


-----

In [9]:

%run initialize-pyegeria.ipynb


----

The python functions are organized into 5 separate clients:

* **EgeriaMy** - for querying an individual's profiles, to-dos and adding feedback to metadata stored in Egeria.
* **EgeriaCat** - for day-to-day use of Egeria
* **EgeriaTech** - for setting up Egeria services and governance actions.
* **EgeriaGov (coming)** - for creating and maintaining governance domains.
* **EgeriaConfig** - for configuring new [Egeria OMAG Servers](https://egeria-project.org/concepts/omag-server/)

`EgeriaCat`, `EgeriaTech` and `EgeriaGov` all extend `EgeriaMy`. `EgeriaConfig` is standalone.

----

----

### EgeriaTech help

This is the call to create the client.

----

In [10]:
egeria_tech = EgeriaTech(view_server, url, user_id, user_pwd)

-----

The next call logs the user into the Egeria platform, a token is returned.  It is retained by the client and automatically passed on each request to Egeria.  The token will eventually time out, at which time just reissue the `create_egeria_bearer_token()` function.

-----

In [11]:
egeria_tech_token = egeria_tech.create_egeria_bearer_token()

----

Now you are ready to use the functions that call Egeria.  It is possible to use the standard python methods to retrieve information about the available functions. For example `dir` lists the functions supported by the client.  

----

In [12]:
dir(egeria_tech)

['__async_get_guid__',
 '__class__',
 '__create_qualified_name__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__get_guid__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__validate_term_status__',
 '__weakref__',
 '_async_activate_server_with_stored_config',
 '_async_add_archive_content',
 '_async_add_archive_file',
 '_async_add_catalog_target',
 '_async_add_comment_reply',
 '_async_add_comment_to_element',
 '_async_add_confidentiality_to_term',
 '_async_add_element_to_subject_area',
 '_async_add_gov_definition_to_element',
 '_async_add_like_to_element',
 '_async_add_ownership_to_element',
 '_async_add_rating_to_element',
 '_async_add_relationship_between_terms',
 '_async_add_subject_area_to

----

A question mark (?) after the name of the client shows how to create an instance of the client.

----

In [13]:
EgeriaTech?

Init signature:
EgeriaTech(
    view_server: str,
    platform_url: str,
    user_id: str,
    user_pwd: str = None,
    token: str = None,
)
Docstring:     
Client for technical Egeria users.

Attributes:

    view_server: str
            Name of the server to use.
    platform_url : str
        URL of the server platform to connect to
    user_id : str
        The identity of the user calling the method - this sets a default optionally used by the methods
        when the user doesn't pass the user_id on a method call.
    user_pwd: str
        The password associated with the user_id. Defaults to None
    token: str, optional
        Bearer token

Methods:
    Inherits methods from EgeriaCat, ActionAuthor, AutomatedCuration,
    ClassificationManager, RegisteredInfo, ValidMetadataManager
File:           /opt/conda/lib/python3.12/site-packages/pyegeria/egeria_tech_client.py
Type:           type
Subclasses:     

----

It is then possible to list the parameters of a specific function using the `help` function.

----

In [14]:
help(EgeriaTech.find_metadata_elements_with_string)

Help on function find_metadata_elements_with_string in module pyegeria.metadata_explorer_omvs:

find_metadata_elements_with_string(self, body: dict, for_lineage: bool = None, for_duplicate_processing: bool = None, start_from: int = 0, page_size: int = 500, time_out: int = 30) -> list | str
    Return a list of metadata elements that match the supplied criteria. The results can be returned over many pages.
    Async version.

    Parameters
    ----------
    body: dict
        - A structure containing the search criteria. (example below)
    for_lineage: bool, default is set by server
        - determines if elements classified as Memento should be returned - normally false
    for_duplicate_processing: bool, default is set by server
        - Normally false. Set true when the caller is part of a deduplication function
    start_from: int, default = 0
        - index of the list to start from (0 for start).
    page_size
        - maximum number of elements to return.
    time_out: int

In [15]:
help(EgeriaTech.get_anchored_metadata_element_graph)

Help on function get_anchored_metadata_element_graph in module pyegeria.metadata_explorer_omvs:

get_anchored_metadata_element_graph(self, guid: str, effective_time: str = None, as_of_time: str = None, for_lineage: bool = None, for_duplicate_processing: bool = None, mermaid_only: bool = False) -> dict | str
    Retrieve the metadata element and all of its anchored elements using its unique identifier. Async version.

    Parameters
    ----------
    guid : str
        - unique identifier of the element to retrieve
    effective_time: str, default = None
        - Time format is "YYYY-MM-DDTHH:MM:SS" (ISO 8601)
    as_of_time: str, default = None
        - Query the element as of this time. If None, then use current time.
    for_lineage: bool, default is set by server
        - determines if elements classified as Memento should be returned - normally false
    for_duplicate_processing: bool, default is set by server
        - Normally false. Set true when the caller is part of a dedu

In [16]:
help(EgeriaTech.get_related_metadata_elements)

Help on function get_related_metadata_elements in module pyegeria.metadata_explorer_omvs:

get_related_metadata_elements(self, guid: str, relationship_type: str, body: dict, for_lineage: bool = None, for_duplicate_processing: bool = None, starting_at_end: int = 0, start_from: int = 0, page_size: int = 500, time_out: int = 30, mermaid_only: bool = False) -> list | str
    Retrieve the metadata elements connected to the supplied element.

    Parameters
    ----------
    guid: str
        - Unique identity of element to retrieve.
    relationship_type: str
        - name of relationship type to retrieve relationships of
    body: dict
        - A structure containing the search criteria. (example below)
    for_lineage: bool, default is set by server
        - determines if elements classified as Memento should be returned - normally false
    for_duplicate_processing: bool, default is set by server
        - Normally false. Set true when the caller is part of a deduplication function
 

In [17]:
help(EgeriaTech.get_asset_graph)

Help on function get_asset_graph in module pyegeria.asset_catalog_omvs:

get_asset_graph(self, asset_guid: str, start_from: int = 0, page_size: int = 500) -> str | dict
    Return all the elements that are anchored to an asset plus relationships between these elements and to
      other elements.
     Parameters
     ----------
     asset_guid : str
         The unique identity of the asset to get the graph for.

     start_from : int, optional
         The index from which to start fetching the engine actions. Default is 0.

     page_size : int, optional
         The maximum number of engine actions to fetch in a single request. Default is `max_paging_size`.

     Returns
     -------
    dict or str
         A dictionary of the asset graph.

     Raises:
     ------
     InvalidParameterException
     PropertyServerException
     UserNotAuthorizedException



In [18]:
help(EgeriaTech.get_asset_lineage_graph)

Help on function get_asset_lineage_graph in module pyegeria.asset_catalog_omvs:

get_asset_lineage_graph(self, asset_guid: str, effective_time: str = None, as_of_time: str = None, relationship_types: [<class 'str'>] = None, limit_to_isc_q_name: str = None, hilight_isc_q_name: str = None, start_from: int = 0, page_size: int = 500) -> str | dict
    Return the asset lineage including a mermaid markdown string. Async Version.
     Parameters
     ----------
     asset_guid : str
         The unique identity of the asset to get the graph for.
    effective_time: str, default is None
        Effective time to query on. If not specified, the current time is used.
    as_of_time: str = None
        as_of_time to query on. If not specified, the current time is used.
    relationship_types: [str], default is None,
        relationship types to include in the lineage graph. If not specified, all relationship types are included.
    limit_to_isc_q_name: str = None,
        if specified, filters r

In [ ]:
help(EgeriaTech.get_servers_by_name)

----

If you execute a function without capturing the response in a python variable, the JSON returned from Egeria is displayed:

----

In [ ]:

egeria_tech.get_servers_by_name("qs-metadata-store")


----

Assigning the results to a python variable means you can extract specific values:

-----

In [ ]:

def get_server_guid(serverName):
    serverDetails = egeria_tech.get_servers_by_name(serverName)
    if serverDetails:
        elementHeader = serverDetails[0].get('elementHeader')
        if elementHeader:
            return elementHeader.get('guid')
    return "???"
            

serverGUID = get_server_guid("qs-integration-daemon")

print("Unique identifier (guid) for 'qs-metadata-store' is " + serverGUID)

In [ ]:
egeria_tech.get_server_report(serverGUID)

----

### EgeriaCat help

This is the call to list the functions for EgeriaCat.

----

In [ ]:
from pyegeria import EgeriaCat

egeria_cat = EgeriaCat(view_server, url, user_id, user_pwd)
egeria_cat_token = egeria_cat.create_egeria_bearer_token()

dir(egeria_cat)

----

Next is the help request for `get_asset_graph`.

-----

In [ ]:

help(EgeriaCat.find_assets_in_domain)


----

### EgeriaConfig help

This is the call to list the functions for EgeriaConfig.

----

In [ ]:
from pyegeria import EgeriaConfig

egeria_config = EgeriaConfig(view_server, url, user_id, user_pwd)
egeria_config_token = egeria_tech.create_egeria_bearer_token()

dir(egeria_tech)

----

Next is the help request for `find_elements_by_property_value`.

-----

In [ ]:

help(EgeriaConfig.get_known_servers)


In [ ]:
from pyegeria.commands.cat.glossary_actions import display_glossaries


In [ ]:
# print("*", view_server, url, user_id, user_pwd)
display_glossaries(search_string="*", view_server = view_server, url = url, uuser_id=user_id, user_pwd = user_pwd)


In [ ]:
help(list_glossaries)

In [ ]:
from pyegeria.commands import display_gov_eng_status
